In [408]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


这个notebook包括所有执行代码，主要逻辑则包括再action_machine.py, event_manager.py, novel_helper.py等几个python文件里

In [2]:
import os
import sys
import random
import math
import collections
import logging
import pandas as pd
import numpy as np

from proto.book_def_pb2 import Book as BookDef
from proto.book_def_pb2 import BookWarehouse
from proto.user_def_pb2 import Robot as RobotDef
from proto.user_def_pb2 import RobotRegiment
from proto.user_def_pb2 import RobotArmy

from novel_helper import NovelHelper
from novel_helper import BookWarehouseDumper
from novel_helper import S

from enlighten_helper import EnlightenHelper

from action_machine import ActionMachine
from event_manager import EventManager

BookWarehouseDumper用于将爬虫抓的语料生成book_def.proto中的格式

In [3]:
# book_warehouse_dumper = BookWarehouseDumper()

从fictiontg网站上抓下来的有3万余篇，book_meta_10t.csv文件存储着其中1万篇小说，这样可以保证候选的稳定性。如果确有需要，可以通过以下方式重新生成这个csv文件。注意这个是双下划线的方法，这是为了防止误操作。这样的好处是之后可以用另外的1万本小说来测试系统的效果，看一些现象是否还能复现。

In [4]:
# book_warehouse_dumper.__gen_10t_csv()

生成book_warehouse，这个过程包括切句切词，1万篇小说要用20分钟以上。结果存储在book_warehouse_1t.bin和book_warehouse_10t.bin两个二进制文件中

In [5]:
# book_warehouse_dumper.dump_book_warehouse()

从.bin文件加载BookWarehouse进内存，默认是1千篇，可以通过参数改成1万篇

In [6]:
book_warehouse_type = '1t'
book_warehouse = NovelHelper.load_book_warehouse(book_warehouse_type)

生成meta.pkl

In [24]:
# from novel_helper import MetaManager
# meta_manager = MetaManager(book_warehouse)
# meta_manager.dump_meta()

 1000

验证meta.pkl

In [26]:
# meta_manager = MetaManager()
# k, c, t = meta_manager.load_meta()
# len(t)
# len(c)
# len(t)
# list(c.keys())

生成NovelHelper，用于辅助统计与词向量训练，需要传入book_warehouse。会同时load之前存好的word_book_count_map和词向量数据

In [365]:
novel_helper = NovelHelper(book_warehouse=book_warehouse)

一次性的fetch所有较高频单词的汉语翻译

In [37]:
# novel_helper._fetch_cn_online()

 2 thirteen 十三                     list index out of range
list index out of range
 2904 murmurs 杂音                                list index out of range
 7920 tern 燕鸥                                      list index out of range
 9594 eventuality 可能性                            

统计每个单词在多少篇小说里出现了，结果存到word_book_count.csv中

In [ ]:
# novel_helper.dump_word_book_count_csv()

训练词向量（如有需要），会先生成训练语料到wv_corpus.txt，进而训练词向量文件到wv.bin，大约用10分钟

In [ ]:
# novel_helper.train_and_dump_wv_bin()

检查词向量效果-相似词

In [378]:
r=novel_helper.get_similar_word('labia')
r

[('vulva', '外阴', 0.9132454788093003),
 ('clitoris', '阴蒂', 0.8908607176277381),
 ('anus', '肛门', 0.8452633226508285),
 ('cleft', '裂口', 0.8370795520179193),
 ('vagina', '阴道', 0.8328158347117528),
 ('glans', '龟头', 0.8287317064412888),
 ('mons', '蒙斯', 0.8268308055750424),
 ('clit', '阴蒂', 0.826666791463789),
 ('rosebud', '玫瑰花蕾', 0.8211361452646154),
 ('areola', '乳晕', 0.8105542467451445),
 ('folds', '褶皱', 0.801549813523631),
 ('mound', '冢', 0.7921795484071403),
 ('fingertips', '指尖', 0.7837864430293402),
 ('pussy', '猫', 0.7793713335409808),
 ('flesh', '肉', 0.7778966859733228),
 ('cockhead', 'ock头', 0.7756206179127518),
 ('cunny', 'un', 0.7745795603657973),
 ('lips', '嘴唇', 0.7735502159188734),
 ('tongue', '舌', 0.7712116591543574),
 ('slit', '狭缝', 0.7702142031980264)]

检查词向量效果-类比词

In [19]:
# r=novel_helper.get_analogy_word('father', 'fuck', 'mother')
# r

查看所有的keyword、category和rating

In [76]:
# keyword_bid_set_map, category_bid_set_map = collections.defaultdict(set), collections.defaultdict(set)
# rating_bid_set_map = collections.defaultdict(set)
# for book in book_warehouse.book:
#     for keyword in book.keyword:
#         keyword_bid_set_map[keyword.lower()].add(book.bid)
#     for category in book.category:
#         category_bid_set_map[category.lower()].add(book.bid)
#     rating_bid_set_map[book.rating].add(book.bid)
# category_book_count_map = {k.lower():len(category_bid_set_map[k]) for k in category_bid_set_map}
# keyword_book_count_map = {k.lower():len(keyword_bid_set_map[k]) for k in keyword_bid_set_map}
# rating_book_count_map = {k.lower():len(rating_bid_set_map[k]) for k in rating_bid_set_map}
# category_data = list(map(lambda x: {'category': x[0].lower(), 'c_book_count': x[1]}, category_book_count_map.items()))
# keyword_data = list(map(lambda x: {'keyword': x[0].lower(), 'k_book_count': x[1]}, keyword_book_count_map.items()))
# rating_data = list(map(lambda x: {'rating': x[0].lower(), 'r_book_count': x[1]}, rating_book_count_map.items()))
# category_data.sort(key=lambda x: x['c_book_count'], reverse=True)
# keyword_data.sort(key=lambda x: x['k_book_count'], reverse=True)
# rating_data.sort(key=lambda x: x['r_book_count'], reverse=True)
# df_category = pd.DataFrame(category_data)
# df_keyword = pd.DataFrame(keyword_data)
# df_rating = pd.DataFrame(rating_data)
# pd.set_option('display.max_rows', 500) # 打印最大行数
# df_category
# # df_rating
# # df_keyword

将带有某些特征的book以人类可读的方式dump到library里

In [80]:
# bids = list(category_bid_set_map['crime punishment'])
# for book in book_warehouse.book:
#     if book.bid in bids:
#         novel_helper.dump_book_to_library(book)

EnlighterHelper，用于生成并enlighten（启蒙）机器人用户大军

In [7]:
enlighten_helper = EnlightenHelper(robot_army = None, novel_helper=novel_helper)

开始生产RobotUser

In [8]:
# enlighten_helper.enlighten_and_dump()

加载训练好的RobotArmy

In [39]:
# robot_army = EnlightenHelper.load_robot_army()

检查机器人军团整体情况

In [18]:
# enlighten_helper.report_robot_army(robot_army)

regiment_count: 10,robot_count: 10000


检查robot信息（机器人秘书）

In [85]:
# robot_army_secret = robot_army.regiment[0].robot[6]
# enlighten_helper.get_robot_repr(robot_army_secret)

初始化一个ActionMachine

In [180]:
action_machine = ActionMachine(novel_helper=novel_helper, robot_army=robot_army)

测试`_get_click_drive_score`,`_get_read_drive_score`、`_get_detest_drive_score`

In [179]:
# book_x = book_warehouse.book[0]
# robot_user_x = robot_army.regiment[0].robot[0]
# chapter = 3
# click_drive_score = action_machine._get_click_drive_score(robot_user_x, book_x, sample_loop=2)
# read_drive_score = action_machine._get_read_drive_score(robot_user_x, book_x, chapter, sample_loop=2)
# detest_title_drive_score = action_machine._get_title_detest_drive_score(robot_user_x, book_x, sample_loop=2)
# detest_drive_score = action_machine._get_detest_drive_score(robot_user_x, book_x, chapter, sample_loop=2)
# addict_drive_score = action_machine._get_addict_drive_score(robot_user_x, book_x, chapter, sample_loop=2)

# print(click_drive_score, read_drive_score, detest_title_drive_score, detest_drive_score, addict_drive_score)

0.06187081848340892 0.02144914949985688 0.01875598491960219 -0.01981412640888699 0.040873518941113046


运行30个period

In [478]:
action_machine = ActionMachine(novel_helper=novel_helper, robot_army=robot_army)
action_machine.run(30)

period: 30,offline: 4519,recommend: 894,choosing: 2952,click_drive: 1116,read_drive: 252,reading: 267          

In [425]:
from IPython.display import IFrame
use_query="offline_uid,recommend_uid,choosing_uid,click_drive_uid,read_drive_uid,reading_uid"
# IFrame('http://localhost:8001?use_query='+use_query, width=800, height=600)

统计点击率

In [443]:
ctr_sql = """select 1.0 * sum(case when t2.uid is not null then 1 else 0 end)/sum(1) as ctr, sum(1) as impr_cnt from
(select uid, bid from novel_event where event = 'impression') t1 left join
(select uid, bid from novel_event where event = 'go_detail') t2 on t1.uid = t2.uid and t1.bid = t2.bid
"""
action_machine.query(ctr_sql, reprepare=False)

[(0.005232341574707827, 171816)]

In [363]:

read_count_per_book_sql = """
select bid, count(*) as read_cnt from novel_event where event = 'go_detail' 
group by bid order by read_cnt desc limit 1000
"""
how_many_book_sql = """
select count(distinct bid) from novel_event
"""
row_list = action_machine.query(read_count_per_book_sql, reprepare=False)
tp_list = []
for row in row_list:
    bid = row[0]
    title = ' '.join(list(action_machine.book_map[bid]._book_def.title.word))
    tp = (row[0], row[1], title)
    tp_list.append(tp)
tp_list

[('1423946587589307172', 60, 'The Price of Marrying Nobility Part 2'),
 ('325621755757732500', 50, 'Absence Makes The Lust'),
 ('34287264511000583002', 50, 'The Softening of Jessie ( Part 32 )'),
 ('127859655679861804', 46, "Trudy 's Story"),
 ('336986046810351352', 46, 'Mistress By Mail'),
 ('3409257891420842390', 40, 'Dreamer , Nothing but a dreamer'),
 ('127168970326450523', 35, 'The Girl/The Boy , Part 3'),
 ('3413599821149761488', 32, 'Atonement'),
 ('33269915831418686456', 27, "Mom 's Unique Punishment Part 2"),
 ('1274199823401742076', 26, 'A Fantasy in Shades of Scarlett'),
 ('1348390261421201042',
  26,
  'ChangeDay : Das erste Mal in Deutschland Teil 8'),
 ('1401503975607934679', 25, 'Camera of Change : Beginnings'),
 ('1515525666140111867', 25, 'Emma The Beginning Part 4'),
 ('3037098169199044252', 25, 'Chatting with Jack Chalker 3 Post-Chat'),
 ('31976486902423322941', 25, 'A Quick History of FM'),
 ('337789402964975315', 24, 'Chrissy the Masseuse Chapter 4'),
 ('3304474989